In [ ]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
"""
# If you're using Google Colab and not running locally, run this cell.

## Install dependencies
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install text-unidecode

# ## Install NeMo
BRANCH = 'main'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

## Install TorchAudio
!pip install torchaudio>=0.10.0 -f https://download.pytorch.org/whl/torch_stable.html

## Grab the config we'll use in this example
!mkdir configs

# minGPT License
*このノートブックでは、[minGPTコードベース](https://github.com/karpathy/minGPT)を同等のNeMoコードに移植しています。したがって、minGPTのライセンスはここに添付されています。*
```
MIT License (MIT) Copyright (c) 2020 Andrej Karpathy
ここに、本ソフトウェアおよび関連文書ファイル（以下「本ソフトウェア」という）の複製を入手した者に対し、制限なく本ソフトウェアを使用するための無償の許可が与えられる。これには、使用、複製、改変、結合、公開、配布、サブライセンス、および／または本ソフトウェアのコピーを販売するための権利が含まれ、また、本ソフトウェアを受領した者に対してもこれらの行為を許可する。ただし以下の条件に従うものとする：
上記の著作権表示および本使用許諾条件は、本ソフトウェアのすべてのコピーまたは重要な部分に含めなければならない。
本ソフトウェアは「現状のまま」提供され、明示的または黙示的を問わず、商品適格性、特定目的への適合性、および非侵害性を含むがこれらに限定されない、あらゆる種類の保証が一切付されていません。いかなる場合においても、著者または著作権者は、契約違反、不法行為その他のいかなる法的理論に基づく場合であっても、本ソフトウェアに関連して生じるいかなる請求、損害またはその他の責任についても、一切の責任を負わないものとします。```

# torch-rnn License*このノートブックでは、[torch-rnn](https://github.com/jcjohnson/torch-rnn) コードベースの `tiny-shakespeare` データセットを使用しています。したがって、torch-rnn のライセンスはここに添付されています。*
```
MITライセンス (MIT)
Copyright (c) 2016 Justin Johnson
ここに、本ソフトウェアおよび関連文書ファイル（以下「本ソフトウェア」という）の複製を入手した者に対し、制限なく本ソフトウェアを使用するための無償の許可が与えられる。これには、使用、複製、改変、結合、公開、配布、サブライセンス、および／または本ソフトウェアのコピーを販売するための権利が含まれ、また、本ソフトウェアを受領した者に対してもこれらの行為を許可する。ただし以下の条件に従うものとする：
上記の著作権表示および本使用許諾条件は、本ソフトウェアのすべてのコピーまたは重要な部分に含めなければならない。
本ソフトウェアは「現状のまま」提供され、明示的または黙示的を問わず、商品適格性、特定目的への適合性、および非侵害性を含むがこれらに限定されない、あらゆる種類の保証が一切付されていません。いかなる場合においても、著者または著作権者は、契約違反、不法行為その他のいかなる法的理論に基づく場合であっても、本ソフトウェアに関連して生じるいかなる請求、損害またはその他の責任についても、一切の責任を負わないものとします。```


-------
***注意: このノートブックでは、`[ERROR CELL]` とマークされたセル内でニューラルタイプやモデル開発の概念の威力を示すため、意図的にエラーを導入します。このようなエラーの説明と解決方法は、後続のセルに記載されています。***
-----

# The NeMo Model
NeMoには、ユーザーが独自のデータセットで迅速にトレーニングとファインチューニングを開始できるよう、複数の最先端の事前学習済み会話型AIモデルが用意されています。
前回の[NeMo入門](https://colab.research.google.com/github/NVIDIA/NeMo/blob/stable/tutorials/00_NeMo_Primer.ipynb)ノートブックでは、NeMoを使用して事前学習済みチェックポイントをダウンロードする方法を学び、NeMoモデルの基本概念についても解説しました。前回のチュートリアルでは、NeMoモデルの使用方法、修正方法、保存方法、および復元方法について説明しました。
このチュートリアルでは、ゼロから非凡なNeMoモデルを開発する方法を学びます。これにより、基盤となるコンポーネントの理解と、それらがPyTorchエコシステム全体とどのように相互作用するかを理解するのに役立ちます。

-------NeMoの中核をなすのは「Model」の概念です。NeMo開発者にとって、「Model」は以下の要素を統合した単一の統一ユニットを指します：
- ニューラルネットワーク自体
- それらのネットワークをサポートするインフラストラクチャ
- これらを単一の統合ユニットとしてパッケージ化したもの
このため、ほとんどのNeMoモデルはデフォルトで以下の機能を備えています（注：一部のNeMoモデルでは、特定のドメイン/ユースケースに特化した追加機能をサポートしています！） - 

[Note: The original text was in Japanese and provided a conceptual explanation of "Model" in NeMo. The English translation maintains the original structure and content while using natural English phrasing.]
- ニューラルネットワークアーキテクチャ - モデルに必要なすべてのモジュール。
-  データセット + データローダー - 訓練や評価時にデータを消費可能な形式に準備するためのすべてのコンポーネント。
-  前処理 + 後処理 - データセットを処理し、モジュールが容易にデータを消費できるようにするコンポーネントのいずれか。
-  Optimizer + Schedulers - デフォルト設定としてすぐに使用可能で、追加の実験も容易に行える基本設定です。
- その他の補助インフラストラクチャ - トークナイザー、言語モデル構成、データ拡張など

# NeMoモデルの構築
NeMoの「モデル」は複数の主要コンポーネントで構成されているため、それらを一つずつ解説していきます。上記の記載順序に従って説明を試みます。
少し難易度を上げるために、今回は自然言語処理分野のモデルを移植してみましょう。Transformerモデルは現在非常に人気があり、BERTやセサミストリート出身の仲間たちは、多くの自然言語処理タスクの中核インフラを形成しています。
NeMoモデルに期待される主要なコンポーネントを簡潔かつ包括的に解説した優れた実装例（ただし簡潔な実装）が、`minGPT`リポジトリで公開されています - https://github.com/karpathy/minGPT。スクリプト自体は短いものの、NeMoモデルに期待される主要なコンポーネントを分かりやすく簡潔に解説しているため、NeMoモデルの開発プロセスを深く理解するための優れた教材と言えます。補足：NeMoはNLPコレクションの一環としてGPTをサポートしており、本ノートブックはこのようなモデルの開発プロセスを深く理解するための詳細な開発解説を目的としています。
以下のノートブックでは、minGPTをNeMoに移植する試みを行い、その過程でNeMo自体の中核的な概念について議論します。

# ニューラルネットワークアーキテクチャの構築
まず第一に、リストの筆頭に来るのはNeMoモデルの中核を成すニューラルネットワークです。
では、このようなモデルをどのように作成するのでしょうか？PyTorchを使用します！以下でご覧いただけるように、NeMoコンポーネントはすべてのPyTorchと互換性があるため、ワークフローを拡張してもPyTorch自体の柔軟性を失うことはありません！
まずはいくつかのインポートから始めましょう：

In [ ]:
import torch
import nemo
from nemo.core import NeuralModule
from nemo.core import typecheck

## ニューラルモジュール待って、`NeuralModule`って何？ あの素晴らしい`torch.nn.Module`はどこ？
`NeuralModule`は`torch.nn.Module`のサブクラスであり、いくつかの追加機能を備えています。
`torch.nn.Module`として完全にPyTorchエコシステムと互換性があるだけでなく、以下の機能を備えています -
1) `Typing` - この機能はモデルに `Neural Type Checking` のサポートを追加します。`Typing` はオプション機能ですが、後述するように非常に有用です！
2) `Serialization` - Remember the `OmegaConf` config dictionary and YAML config files? Well, all `NeuralModules` inherently supports serialization/deserialization from such config dictionaries!
3) `FileIO` - これは完全にオプションのファイルシリアライゼーションシステムです。`NeuralModule` が PyTorch チェックポイントに保存できないデータを保存する方法を必要としている場合、シリアライゼーションとデシリアライゼーションのロジックを2つの便利なメソッドに記述してください！**注意**: 最終的なNeMoモデルを作成する際には、この処理は自動的に実装されます！NeMoモデルの自動シリアライゼーション/デシリアライゼーションサポート！

In [ ]:
class MyEmptyModule(NeuralModule):

  def forward(self):
    print("Neural Module ~ hello world!")

In [ ]:
x = MyEmptyModule()
x()

## ニューラルタイプ
ニューラルタイプ？その用語が何を指すのか疑問に思っているかもしれませんね。
ほぼすべてのNeMoコンポーネントはクラス`Typing`を継承しています。`Typing`はシンプルなクラスで、継承元クラスに2つのプロパティを追加します：`input_types`と`output_types`。最も簡潔に定義すると、NeuralTypeは単なる意味的テンソルです。テンソルが保持すべき意味的形状に関する情報と、そのテンソルが表す意味的情報を含んでいます。以上です。
では、このような型付きテンソルにはどのような意味情報が含まれているのでしょうか？以下に具体例を示します。



------深層学習分野では、テンソルの形状は一致しているにもかかわらず、意味論的には全く一致していないケースにしばしば遭遇します。例えば以下のランク3テンソルをご覧ください -

In [ ]:
# Case 1:
embedding = torch.nn.Embedding(num_embeddings=10, embedding_dim=30)
x = torch.randint(high=10, size=(1, 5))
print("x :", x)
print("embedding(x) :", embedding(x).shape)

In [ ]:
# Case 2
lstm = torch.nn.LSTM(1, 30, batch_first=True)
x = torch.randn(1, 5, 1)
print("x :", x)
print("lstm(x) :", lstm(x)[0].shape)  # Let's take all timestep outputs of the LSTM

-------ご覧の通り、ケース1の出力は形状が[1, 5, 30]の埋め込み表現であり、ケース2の出力はLSTMの出力状態`h`（全時間ステップにわたる）であり、これも同じ形状[1, 5, 30]です。
それらは同じ形状ですか？ **はい**。<br>
Case1 .shape == Case2 .shape を比較した場合、出力結果は True になりますか？ **はい**。<br>それらは同じ概念を表していますか？ **いいえ**。<br>

これら2つのテンソルが同じ意味情報を表現していないことを認識できる能力こそが、私たちがニューラルタイプを活用する理由です。これにはテンソルが表現する形状情報と、その意味概念の両方が含まれています。これら2つのテンソルの出力間でニューラルタイプチェックを行った場合、それらが意味的に異なるものであるというエラーが発生します（より技術的に言えば、それらは互いに`INCOMPATIBLE`であると判定されます）！

--------
[Named Tensors](https://pytorch.org/docs/stable/named_tensor.html) のような概念について聞いたことがあるかもしれません。概念的には似ていますが、NeMoが付与するNeural TypesはPyTorchエコシステムとそれほど緊密に結びついていません。実際、クラスの任意のオブジェクトにNeural Typeを付与することができます！

## ニューラルタイプ - 使用法
ニューラルタイプは興味深い概念ですが、実際に実装するにはどうすればよいでしょうか？以下にいくつかのケースを考えてみましょう。
ニューラルタイプはNeMoの中核的な基盤の一つであり、大多数のニューラルモジュールで使用され、すべてのNeMoモデルではニューラルタイプが定義されます。これらは完全にオプションであり侵入的ではありませんが、NeMoはコンポーネント間の意味的な互換性を確保するためにそれを強力にサポートしています。

型チェックされたモジュールの基本的な例から始めましょう。

In [ ]:
from nemo.core.neural_types import NeuralType
from nemo.core.neural_types import *

In [ ]:
class EmbeddingModule(NeuralModule):
  def __init__(self):
    super().__init__()
    self.embedding = torch.nn.Embedding(num_embeddings=10, embedding_dim=30)

  @typecheck()
  def forward(self, x):
    return self.embedding(x)

  @property
  def input_types(self):
    return {
        'x': NeuralType(axes=('B', 'T'), elements_type=Index())
    }

  @property
  def output_types(self):
    return {
        'y': NeuralType(axes=('B', 'T', 'C'), elements_type=EmbeddedTextType())
    }

ニューラルタイプの有用性を示すため、上記のケースをNeuralModules内で再現します。
上記クラスに型チェック機能を追加する方法について議論しましょう。
1) `forward` には型チェック用のデコレータ `@typecheck()` が付与されています。
2) `input_types` と `output_types` プロパティが定義されています。
それで終わりです。

-------
上記の各ステップについて詳しく説明しましょう。
- `@typecheck()` はシンプルなデコレータで、`Typing` を継承した任意のクラス (NeuralModule が自動的にこれを処理します) に対して、デフォルトで `input_types` と `output_types` という2つのデフォルトプロパティを追加します。これらのプロパティはデフォルトで None を返します。
`@typecheck()` デコレータを明示的に使用することで、デフォルトではニューラル型チェックが**無効化**されます。NeMo はモデル開発プロセスに干渉することを望んでいません。そのため、ユーザーは 2 つのプロパティをオーバーライドすることで型チェックを「オプトイン」できます。したがって、このデコレータは、ユーザーが型チェックを必要としない限り、型チェックによって負担を強いられることがないようにします。
`@typecheck()` とは一体何でしょうか？端的に言えば、これは `Typing` を継承したクラスの任意の関数をラップするために使用できるデコレータです。これにより、そのクラスの型定義を自動的に参照し、強制適用します。通常、`torch.nn.Module` のサブクラスでは通常 `forward()` メソッドしか実装しないため、このメソッドをラップするのが一般的ですが、`@typecheck()` は非常に柔軟なデコレータです。NeMo では、特定のドメイン（TTS など）で非常に重要な高度な使用例をいくつか紹介します。

------
前述の通り、`@typecheck()` は型強制を行います。では、NeMo にこのような型情報をどのように提供すればよいのでしょうか？
クラスの `input_types` と `output_types` プロパティをオーバーライドすることで、文字列名を `NeuralType` にマッピングする辞書を返すことができます。
上記の場合、`NeuralType`を以下の2つの要素で定義します：
- `axes`: これは軸自体が持つ意味情報です。最も一般的な軸情報は単一文字表記によるものです。
> `B` = バッチ<br>> `C` / `D` - チャネル / 次元（同一扱い） <br>> `T` - 時間 <br>> `H` / `W` - 高さ / 幅 <br>
- `elements_type`: これは「テンソルが表すもの」の意味情報です。すべてのこのような型は基本クラス `ElementType` から派生しており、単に `ElementType` をサブクラス化するだけで、NeMo で使用可能なカスタム意味型の階層構造を構築できます！
ここで、入力は `Index` 型の `ElementType` (語彙内の文字のインデックス) であり、出力は `EmbeddedTextType` 型の `ElementType` (テキスト埋め込み) であると宣言します。

In [ ]:
embedding_module = EmbeddingModule()

次に、上記のケース2に相当するものを`NeuralModule`として構築してみましょう。

In [ ]:
class LSTMModule(NeuralModule):
  def __init__(self):
    super().__init__()
    self.lstm = torch.nn.LSTM(1, 30, batch_first=True)

  @typecheck()
  def forward(self, x):
    return self.lstm(x)

  @property
  def input_types(self):
    return {
        'x': NeuralType(axes=('B', 'T', 'C'), elements_type=SpectrogramType())
    }

  @property
  def output_types(self):
    return {
        'y': NeuralType(axes=('B', 'T', 'C'), elements_type=EncodedRepresentation())
    }

------ここでは、上記ケース2のLSTMモジュールを定義します。
入力をランク3のテンソルに変更し、これは現在「スペクトログラムタイプ」を表現しています。意図的に汎用的に設計しており、入力として`MelSpectrogramType`や`MFCCSpectrogramType`としても使用可能です！
LSTMの出力は現在`EncodedRepresentation`となっています。実際には、CNN層の出力、Transformerブロックの出力、あるいはこの場合はLSTM層の出力として扱うことができます。もちろん、EncodedRepresentationをサブクラス化して独自の実装を作成することも可能です！

In [ ]:
lstm_module = LSTMModule()

------さあ、テストを始めましょう！

In [ ]:
# Case 1 [ERROR CELL]
x1 = torch.randint(high=10, size=(1, 5))
print("x :", x1)
print("embedding(x) :", embedding_module(x1).shape)

-----なぜ最初から `TypeError` が発生するのか疑問に思うかもしれません。この `TypeError` は意図的に発生するように設計されています。
位置引数はモデル開発時に重大な問題を引き起こす可能性があり、特にモデル/モジュール設計が確定していない場合に問題が発生しやすくなります。誤った位置引数によるエラーの可能性を減らし、関数に渡される引数の名前を強制するために、`Typing`では**すべての型チェック済み関数をkwargsのみを使用して呼び出すこと**を要求しています。

In [ ]:
# Case 1
print("x :", x1)
print("embedding(x) :", embedding_module(x=x1).shape)

では、ケース2の`LSTMModule`についても同様に試してみましょう。

In [ ]:
# Case 2 [ERROR CELL]
x2 = torch.randn(1, 5, 1)  # Input = [B=1, T=5, C=1]
print("x :", x2)
print("lstm(x) :", lstm_module(x=x2)[0].shape)  # Let's take all timestep outputs of the LSTM

-----今度は、提供された出力引数の数が期待される数と一致しないという型エラーが発生します。
ここで具体的に何が行われているのでしょうか？ 私たちの `LSTMModule` クラス内部では、出力タイプを単一のニューラルタイプ - 形状が [B, T, C] の `EncodedRepresentation` と宣言しています。
しかし、LSTM層の出力は以下のようなタプルです：1) [B, T, C] 形状のエンコード表現2) もう一つのタプルで、隠れ状態`h`とセル状態`c`の2つの状態値を含んでいます。それぞれの形状は[num_layers * num_directions, B, C]です！
このため、ニューラル型システムは出力引数の数が期待される値と一致しないことを示すエラーを発生させます。
**注**: 2つの状態の軸タイプ情報は `D` で表されます。これは一般的な「次元」を表すためです。`num_layers` と `num_directions` は単一の軸に統合されているためです。NeMoにおいて、`C` と `D` の軸タイプは同等であり、相互に置き換え可能であるため、ここでは `C` を使用してLSTMの隠れ次元を表し、`D` を使用して統合された軸 `num_layers * num_directions` を表します。
上記を修正しましょう。

In [ ]:
class CorrectLSTMModule(LSTMModule):  # Let's inherit the wrong class to make it easy to override
  @property
  def output_types(self):
    return {
        'y': NeuralType(axes=('B', 'T', 'C'), elements_type=EncodedRepresentation()),
        'h_c': [NeuralType(axes=('D', 'B', 'C'), elements_type=EncodedRepresentation())],
    }

`h_c` ニューラルタイプについては、リスト `[]` でラップする必要があります。NeMo はデフォルトで、各 `NeuralType` が単一の戻り値に対応すると仮定しています。ただし、LSTM の場合、2つの状態テンソルからなるタプルを生成します。
そこで、NeMoに対してこの特定の`NeuralType`が単一次元のアイテムリストであり、このリストの各要素が同じ`NeuralType`を持ち、同じ形状であることを通知します。
NeMoはその後、`h_c`が常にテンソルのリストであることを保証します。リスト内のアイテムの数はチェックしませんが、返される値が*ゼロ個以上のアイテムを含むリスト*であること、そしてこれらの各アイテムが同じ`NeuralType`を共有していることを保証します。

In [ ]:
lstm_module = CorrectLSTMModule()

In [ ]:
# Case 2
x2 = torch.randn(1, 5, 1)
y2, (h, c) = lstm_module(x=x2)
print("x :", x2)
print("lstm(x) :", y2.shape)  # The output of the LSTM RNN
print("hidden state (h) :", h.shape)  # The first hidden state of the LSTM RNN
print("hidden state (c) :", c.shape)  # The second hidden state of the LSTM RNN

------素晴らしい！これで型チェックシステムは満足しています。
よく見ると、これらの出力は通常のTorchテンソルです（これは良いニュースです。結局のところ、Torchテンソルと互換性を持たせたくないですからね！）では、具体的にどのような情報が格納されているのでしょうか？
`output_types`がオーバーライドされ、有効なtorchテンソルが結果として返される場合、これらのテンソルには`neural_type`属性が付加されます。これを確認してみましょう -

In [ ]:
emb_out = embedding_module(x=x1)
lstm_out = lstm_module(x=x2)[0]

assert hasattr(emb_out, 'neural_type')
assert hasattr(lstm_out, 'neural_type')

In [ ]:
print("Embedding tensor :", emb_out.neural_type)
print("LSTM tensor :", lstm_out.neural_type)

-------このように、これらのテンソルには新たに `neural_type` という属性が追加され、形状も同じになっています。
この演習の目的は、たとえ形状が同じであっても、二つの出力が意味的に**同一のオブジェクトではない**ことを主張することにありました。
これをテストしてみよう！

In [ ]:
emb_out.neural_type.compare(lstm_out.neural_type)

In [ ]:
emb_out.neural_type == lstm_out.neural_type

## ニューラルタイプ - 制約事項
興味深いことに、私たちの入力は型付き関数呼び出しの両方において単なる `torch.Tensor` であり、それらには `neural_type` が割り当てられていませんでした。
では、型チェックシステムはなぜエラーを検出しなかったのでしょうか？
これは互換性を維持するためです - 型チェックは関数呼び出しの連鎖において機能するように設計されています - そして各関数自体も `@typecheck()` デコレータでラップされるべきです。これはまた、フォワードコールに数十ものチェックを課すことを避けるためであり、したがって私たちは高次の論理計算を実行するモジュールのみを型付けします。
------
具体例として、ResNetモデルの各残差ブロックの入力と出力を個別に記述する必要はほとんどありません（ただし可能です）。しかし、エンコーダ（内部の層数にかかわらず）およびデコーダ（分類ヘッド）を個別に記述することは実用的に重要です。これにより、ファインチューニングを行う際に、エンコーダに入力されるテンソルとデコーダにバインドされるテンソルの間に意味的な不一致が生じないようにできます。

-------このケースでは、クラスを拡張して入力テンソルに型を付加するのは非現実的であるため、ショートカットとして直接入力テンソルにニューラル型を付加することができます！

In [ ]:
embedding_module = EmbeddingModule()
x1 = torch.randint(high=10, size=(1, 5))

# Attach correct neural type
x1.neural_type = NeuralType(('B', 'T'), Index())

print("embedding(x) :", embedding_module(x=x1).shape)

In [ ]:
# Attach wrong neural type [ERROR CELL]
x1.neural_type = NeuralType(('B', 'T'), LabelsType())

print("embedding(x) :", embedding_module(x=x1).shape)

## minGPTコンポーネントを作成しよう
ニューラル型チェックについてある程度理解が深まったところで、minGPTのサンプルコードの移植を開始しましょう。ここでも、コードの大部分は[minGPTリポジトリ](https://github.com/karpathy/minGPT)からそのまま移植する形で進めます。
ここで注目すべき点があります。クラスインポートを変更するだけで、`@typecheck()`を前方に適用し、`input_types`と`output_types`を追加するだけで（これらは完全にオプションです！）、PyTorch Lightningへの移植作業はほぼ完了です！
**注意**: GPTコンポーネントクラスをすべてヘルパーモジュールに移動しました。これは、NeMoのセキュリティ検証プロセスにおける`__main__`名前空間の問題を回避するためです。以下の方法でインポートしてください:

In [ ]:
from helper_files.gpt_components import (
    AttentionType, SelfAttentionType, CausalSelfAttentionType,
    CausalSelfAttention, Block,
    GPTEmbedding, GPTTransformerEncoder, GPTDecoder
)


In [ ]:
# Basic imports needed for the tutorial
import math
from typing import List, Set, Dict, Tuple, Optional

import torch
import torch.nn as nn
from torch.nn import functional as F

## 要素タイプの作成
これまで私たちは NeMo コアが提供するニューラルタイプを使用してきました。しかし、事前に定義された要素タイプに限定される必要はありません！
ユーザーは自由に、任意の階層構造を持つ要素タイプを定義できます！
補助モジュールでは、注意機構関連のニューラルタイプの階層構造を作成する `AttentionType`、`SelfAttentionType`、`CausalSelfAttentionType` といったカスタム要素タイプを定義しています。これらのタイプは `helper_files.gpt_components` からインポートされています。

In [ ]:
# Custom element types are now imported from helper_files.gpt_components:
# - AttentionType(EncodedRepresentation): Basic Attention Element Type
# - SelfAttentionType(AttentionType): Self Attention Element Type
# - CausalSelfAttentionType(SelfAttentionType): Causal Self Attention Element Type
print("Custom element types imported successfully!")

## モジュールの作成
ニューラルモジュールは通常最上位レベルのモジュールですが、モジュール階層の任意のレベルで使用することができます。
デモンストレーションとして、因果自己注意モジュールのブロックから構成されるエンコーダを型付きニューラルモジュールとして扱います。もちろん、各因果自己注意層そのものをニューラルモジュールとして扱うことも可能ですが、一般的には最上位モジュールが好まれます。
基本的な PyTorch モジュール (`CausalSelfAttention` と `Block`) は、NeMo のセキュリティ検証で発生する `__main__` 名前空間の問題を回避するため、ヘルパーモジュールからインポートされています。

In [ ]:
# CausalSelfAttention and Block classes are now imported from helper_files.gpt_components
# These are standard PyTorch nn.Module implementations:
# - CausalSelfAttention: A vanilla multi-head masked self-attention layer
# - Block: An unassuming Transformer block combining attention and MLP

print("Basic PyTorch modules imported successfully!")
print(f"CausalSelfAttention: {CausalSelfAttention}")
print(f"Block: {Block}")

## NeMoモデルの構築
NeMoモデルは様々なコンポーネントで構成されているため、このノートブック内で段階的にモデルを構築していきます。その結果、複数の中間段階のNeMo「モデル」が生成され、これらは部分的な実装となり、相互に反復的に継承されることになります。
NeMoモデル（NeMoコレクションに含まれるもの）を完全に実装する場合、これらのコンポーネントは通常単一のクラス内に配置されます。
まず、PyTorch NeMo Modelの中核クラスである`ModelPT`を継承しましょう。これはPyTorch Lightning Moduleを継承したクラスです。

-------**Remember**:
- `torch.nn.Module` に相当する NeMo のクラスは `NeuralModule` です。- `LightningModule`に相当する`ModelPT`クラスが用意されています。

In [ ]:
import lightning.pytorch as ptl
from nemo.core import ModelPT
from omegaconf import OmegaConf

------次に、NeMoモデルの最小限の実装を構築しましょう。具体的にはコンストラクタ、重みの初期化メソッド、およびforwardメソッドを実装します。
まずはminGPT実装で採用されている手順に従い、その後NeMo向けに段階的にリファクタリングを行う。

In [ ]:
class PTLGPT(ptl.LightningModule):
  def __init__(self,
                 # model definition args
                 vocab_size: int, # size of the vocabulary (number of possible tokens)
                 block_size: int, # length of the model's context window in time
                 n_layer: int, # depth of the model; number of Transformer blocks in sequence
                 n_embd: int, # the "width" of the model, number of channels in each Transformer
                 n_head: int, # number of heads in each multi-head attention inside each Transformer block
                 # model optimization args
                 learning_rate: float = 3e-4, # the base learning rate of the model
                 weight_decay: float = 0.1, # amount of regularizing L2 weight decay on MatMul ops
                 betas: Tuple[float, float] = (0.9, 0.95), # momentum terms (betas) for the Adam optimizer
                 embd_pdrop: float = 0.1, # \in [0,1]: amount of dropout on input embeddings
                 resid_pdrop: float = 0.1, # \in [0,1]: amount of dropout in each residual connection
                 attn_pdrop: float = 0.1, # \in [0,1]: amount of dropout on the attention matrix
                 ):
        super().__init__()

        # save these for optimizer init later
        self.learning_rate = learning_rate
        self.weight_decay = weight_decay
        self.betas = betas

        # input embedding stem: drop(content + position)
        self.tok_emb = nn.Embedding(vocab_size, n_embd)
        self.pos_emb = nn.Parameter(torch.zeros(1, block_size, n_embd))
        self.drop = nn.Dropout(embd_pdrop)
        # deep transformer: just a sequence of transformer blocks
        self.blocks = nn.Sequential(*[Block(n_embd, block_size, n_head, attn_pdrop, resid_pdrop) for _ in range(n_layer)])
        # decoder: at the end one more layernorm and decode the answers
        self.ln_f = nn.LayerNorm(n_embd)
        self.head = nn.Linear(n_embd, vocab_size, bias=False) # no need for extra bias due to one in ln_f

        self.block_size = block_size
        self.apply(self._init_weights)

        print("number of parameters: %e" % sum(p.numel() for p in self.parameters()))

  def forward(self, idx):
      b, t = idx.size()
      assert t <= self.block_size, "Cannot forward, model block size is exhausted."

      # forward the GPT model
      token_embeddings = self.tok_emb(idx) # each index maps to a (learnable) vector
      position_embeddings = self.pos_emb[:, :t, :] # each position maps to a (learnable) vector
      x = self.drop(token_embeddings + position_embeddings)
      x = self.blocks(x)
      x = self.ln_f(x)
      logits = self.head(x)

      return logits

  def get_block_size(self):
      return self.block_size

  def _init_weights(self, module):
      """
      Vanilla model initialization:
      - all MatMul weights \in N(0, 0.02) and biases to zero
      - all LayerNorm post-normalization scaling set to identity, so weight=1, bias=0
      """
      if isinstance(module, (nn.Linear, nn.Embedding)):
          module.weight.data.normal_(mean=0.0, std=0.02)
          if isinstance(module, nn.Linear) and module.bias is not None:
              module.bias.data.zero_()
      elif isinstance(module, nn.LayerNorm):
          module.bias.data.zero_()
          module.weight.data.fill_(1.0)

------念のため、上記のPyTorch Lightningモデルを作成して動作を確認しましょう !

In [ ]:
m = PTLGPT(vocab_size=100, block_size=32, n_layer=1, n_embd=32, n_head=4)

------それでは、上記のコードを簡単にNeMoモデルに変換してみましょう。
NeMoモデルコンストラクタは通常、以下の2つの要素のみを受け入れます：
1) `cfg`: OmegaConf DictConfigオブジェクトで、モデルがニューラルネットワークアーキテクチャ、データローダー設定、オプティマイザ設定、およびモデル自体に必要なその他のコンポーネントを定義するために使用する正確な構成要素を指定します。
2) `trainer`: PyTorch LightningのオプションのTrainerオブジェクトで、NeMoモデルをトレーニングに使用する場合に指定できます。構築後（必要に応じて）に`set_trainer`メソッドを使用して設定できます。このノートブックでは、Trainerオブジェクトの設定は行いません。

## ニューラルモジュールのリファクタリング
前述の通り、ニューラルモジュールは一般にモデルの上位レベル構成要素であり、同等のニューラルモジュールで置き換えることが可能である。
上述の通り、埋め込みモジュール、深層トランスフォーマーデコーダネットワーク、および最終デコーダ層はすべて、PyTorch Lightning実装のコンストラクタ内で統合されています。
------
ただし、最終デコーダーモジュールは単純な線形層ではなく、RNNで構成されていた可能性もあります。あるいは、1次元CNNが採用されていた可能性もあります。
同様に、深層トランスフォーマーデコーダでは、Self Attentionモジュールの実装が異なる可能性がある。
これらの変更は、上記の実装内で直接実装するのは容易ではありません。しかし、これらのコンポーネントをそれぞれ独立したNeuralModuleにリファクタリングすれば、将来的に構築する同等のモジュールで簡単に置き換えることが可能になります！

### 埋め込みモジュールのリファクタリング
まず、上記の実装から埋め込みモジュールをリファクタリングしましょう。`GPTEmbedding`クラスは、ヘルパーモジュールからインポートされています。

In [ ]:
# GPTEmbedding NeuralModule is now imported from helper_files.gpt_components
# It implements token and positional embeddings with dropout
print(f"GPTEmbedding imported: {GPTEmbedding}")

# Example instantiation (with dummy parameters for demonstration)
dummy_embedding = GPTEmbedding(vocab_size=100, n_embd=32, block_size=128)
print(f"Input types: {dummy_embedding.input_types}")
print(f"Output types: {dummy_embedding.output_types}")

### エンコーダのリファクタリング
次に、GPTエンコーダのリファクタリングを行います。これはマルチレイヤーTransformer（Decoder）ネットワークとして実装されています。`GPTTransformerEncoder`クラスは、現在補助モジュールからインポートされています。
------ここで言及しているのはGPTの「Encoder」モジュールですが、実際にはTransformerの「Decoder」ブロックを用いて構築されています。
***「ニューラルモジュール」について議論する際、我々は特定の入力ニューラルタイプと特定の出力ニューラルタイプを備えた抽象的なモジュールについて議論しているのである。***
私たちにとって、GPT「Encoder」ニューラルモジュールは、任意の実装を受け入れるように設計されており、その
- 入力ニューラルタイプは `NeuralType(('B', 'T', 'C'), EmbeddedTextType())` です
- 出力型は `NeuralType(('B', 'T', 'C'), EncodedRepresentation())` です
-----このようなGPT「エンコーダ」ニューラルモジュールの具体的な実装例として、Deep Transformer「デコーダ」ネットワークが挙げられる。

In [ ]:
# GPTTransformerEncoder NeuralModule is now imported from helper_files.gpt_components
# It implements a sequence of transformer blocks for encoding
print(f"GPTTransformerEncoder imported: {GPTTransformerEncoder}")

# Example instantiation (with dummy parameters for demonstration)
dummy_encoder = GPTTransformerEncoder(n_embd=32, block_size=128, n_head=4, n_layer=1)
print(f"Input types: {dummy_encoder.input_types}")
print(f"Output types: {dummy_encoder.output_types}")

### デコーダのリファクタリング
最後に、解答をデコードするための小規模な単一層フィードフォワードネットワークであるDecoderをリファクタリングしましょう。`GPTDecoder`クラスは、ヘルパーモジュールからインポートされています。
-------
注目すべき点として、Decoderの`input_types`は汎用的な`EncoderRepresentation()`を受け入れますが、`GPTTransformerEncoder`の`neural_type`は`output_type`として`CausalSelfAttentionType`を採用しています。
これは意味論的にはミスマッチではありません！上記の継承関係図でご覧いただけるように、`EncodedRepresentation` -> `AttentionType` -> `SelfAttentionType` -> `CausalSelfAttentionType` という継承関係を宣言しています。
この`element_type`の継承階層により、将来のエンコーダ（少なくとも`EncodedRepresentation`型のニューラル出力を持つエンコーダ）が、現在のGPT因果自己注意エンコーダと置き換え可能になります。これにより、NeMoモデルのその他の部分は問題なく動作し続けます！

In [ ]:
# GPTDecoder NeuralModule is now imported from helper_files.gpt_components
# It implements layer normalization followed by a linear layer to produce logits
print(f"GPTDecoder imported: {GPTDecoder}")

# Example instantiation (with dummy parameters for demonstration)
dummy_decoder = GPTDecoder(n_embd=32, vocab_size=100)
print(f"Input types: {dummy_decoder.input_types}")
print(f"Output types: {dummy_decoder.output_types}")


### NeMo GPTモデルのリファクタリング
埋め込み層、エンコーダ、デコーダ用にそれぞれ3つのNeuralModuleを実装したので、このリファクタリングを活用してNeMoモデルを再構成しましょう！
今回は汎用の`LightningModule`ではなく、`ModelPT`を継承しています。

In [ ]:
class AbstractNeMoGPT(ModelPT):
  def __init__(self, cfg: OmegaConf, trainer: ptl.Trainer = None):
      super().__init__(cfg=cfg, trainer=trainer)

      # input embedding stem: drop(content + position)
      self.embedding = self.from_config_dict(self.cfg.embedding)
      # deep transformer: just a sequence of transformer blocks
      self.encoder = self.from_config_dict(self.cfg.encoder)
      # decoder: at the end one more layernorm and decode the answers
      self.decoder = self.from_config_dict(self.cfg.decoder)

      self.block_size = self.cfg.embedding.block_size
      self.apply(self._init_weights)

      print("number of parameters: %e" % self.num_weights)

  @typecheck()
  def forward(self, idx):
      b, t = idx.size()
      assert t <= self.block_size, "Cannot forward, model block size is exhausted."

      # forward the GPT model
      # Remember: Only kwargs are allowed !
      e = self.embedding(idx=idx)
      x = self.encoder(embed=e)
      logits = self.decoder(encoding=x)

      return logits

  def get_block_size(self):
      return self.block_size

  def _init_weights(self, module):
      """
      Vanilla model initialization:
      - all MatMul weights \in N(0, 0.02) and biases to zero
      - all LayerNorm post-normalization scaling set to identity, so weight=1, bias=0
      """
      if isinstance(module, (nn.Linear, nn.Embedding)):
          module.weight.data.normal_(mean=0.0, std=0.02)
          if isinstance(module, nn.Linear) and module.bias is not None:
              module.bias.data.zero_()
      elif isinstance(module, nn.LayerNorm):
          module.bias.data.zero_()
          module.weight.data.fill_(1.0)

  @property
  def input_types(self):
    return {
        'idx': NeuralType(('B', 'T'), Index())
    }

  @property
  def output_types(self):
    return {
        'logits': NeuralType(('B', 'T', 'C'), LogitsType())
    }

## モデル用の設定ファイルを作成する
一見すると、上記のPyTorch Lightning実装と比べて特に変更はありません。コンストラクタがconfigオブジェクトを受け入れるようになった点を除いて、まったく変更はありません！
NeMoモデルは、対応する設定辞書（OmegaConfオブジェクトとしてインスタンス化）と共に動作します。この仕組みにより、Hydraを迅速に活用してモデルのプロトタイプ作成が可能になります。その他にも、ハイパーパラメータ最適化やNeMoモデルのシリアライズ/デシリアライズなど、様々な利点があります。
実際にこのような設定オブジェクトを構築する方法を見てみましょう！

In [ ]:
# model definition args (required)
# ================================
# vocab_size: int # size of the vocabulary (number of possible tokens)
# block_size: int # length of the model's context window in time
# n_layer: int # depth of the model; number of Transformer blocks in sequence
# n_embd: int # the "width" of the model, number of channels in each Transformer
# n_head: int # number of heads in each multi-head attention inside each Transformer block

# model definition args (optional)
# ================================
# embd_pdrop: float = 0.1, # \in [0,1]: amount of dropout on input embeddings
# resid_pdrop: float = 0.1, # \in [0,1]: amount of dropout in each residual connection
# attn_pdrop: float = 0.1, # \in [0,1]: amount of dropout on the attention matrix

------上記の必須パラメータを確認すると、OmegaConfに対してこれらの値が現在設定されていないが、ユーザーが使用する前に設定すべきであることを示す方法が必要です。
OmegaConf では `MISSING` 値を使用してこのような動作をサポートしています。YAML 設定ファイルでは、プレースホルダーとして `???` を使用することで同様の効果が得られます。

In [ ]:
from omegaconf import MISSING

In [ ]:
# Let's create a utility for building the class path
def get_class_path(cls):
  return f'{cls.__module__}.{cls.__name__}'

### Model configの構造
まず、モデルレベルの設定ファイルで使用する共通コンポーネントの設定を作成しましょう：

In [ ]:
common_config = OmegaConf.create({
    'vocab_size': MISSING,
    'block_size': MISSING,
    'n_layer': MISSING,
    'n_embd': MISSING,
    'n_head': MISSING,
})

-----現在のモデル設定はまだ構築中です。もっと詳細な情報を追加する必要があります！
完全な Model Config には、最上位モジュールのすべてのサブ構成も含める必要があります。具体的には、`embedding`、`encoder`、および `decoder` の各構成が含まれます。

### サブモジュール設定の構造
最上位モデルの場合、実際のモジュール自体を変更することはほとんどなく、代わりにそのモデルのハイパーパラメータを主に変更します。
そこで、`Hydra`のクラスインスタンス化メソッドを活用します。このメソッドは、クラスメソッド`ModelPT.from_config_dict()`を通じて簡単にアクセスできます。
以下にいくつかの具体例を挙げましょう -

In [ ]:
embedding_config = OmegaConf.create({
    '_target_': get_class_path(GPTEmbedding),
    'vocab_size': '${model.vocab_size}',
    'n_embd': '${model.n_embd}',
    'block_size': '${model.block_size}',
    'embd_pdrop': 0.1
})

encoder_config = OmegaConf.create({
    '_target_': get_class_path(GPTTransformerEncoder),
    'n_embd': '${model.n_embd}',
    'block_size': '${model.block_size}',
    'n_head': '${model.n_head}',
    'n_layer': '${model.n_layer}',
    'attn_pdrop': 0.1,
    'resid_pdrop': 0.1
})

decoder_config = OmegaConf.create({
    '_target_': get_class_path(GPTDecoder),
    # n_embd: int, vocab_size: int
    'n_embd': '${model.n_embd}',
    'vocab_size': '${model.vocab_size}'
})

##### `_target_`とは何ですか？--------
上記の設定では、configファイル内に`_target_`が指定されています。`_target_`は通常、Pythonパッケージまたはユーザーローカルディレクトリ内の実際のクラスへの完全なクラスパスを指定します。Hydraがこのパスからモデルを正しく検索してインスタンス化するために必要です。
では、なぜクラスパスを設定したいのでしょうか？
一般的にモデル開発においては、エンコーダやデコーダ自体を変更することはあまりありませんが、エンコーダとデコーダのハイパーパラメータを変更することはよく行われます。
この表記法は、モデルレベルの順方向処理の宣言を簡潔かつ正確に記述するのに役立ちます。また、モデルのどの部分を容易に置き換えられるかを論理的に示すこともできます。将来的には、エンコーダを他のタイプの自己注意ブロックに、あるいはデコーダをRNNや1D-CNNニューラルモジュールに容易に置き換えることが可能です（ただし、これらのモジュールは現在のブロックと同じNeural Type定義を持っている必要があります）。

##### `${}` 構文とは何ですか？-------
OmegaConf、および間接的にHydraも変数補間をサポートしています。埋め込み層、エンコーダ、デコーダのニューラルモジュールの`__init__`メソッドでご覧いただけるように、これらのモジュール間では多くのパラメータが共有されています。
各埋め込み層、エンコーダ、デコーダの設定でこれらのコンストラクタの値を個別に設定するのは、煩雑でエラーが発生しやすい作業になります。
代わりに、`model`レベルの設定内で標準キーを定義し、それぞれの設定内でこれらの値を補間します！

### モデルとモジュールレベルの設定の添付
現在、コアコンポーネントにはモデルレベルとモジュールごとの設定が用意されています。サブモジュールの設定は通常「model」名前空間に属しますが、必要に応じて独自の構造を定義することも可能です。
それらを貼り付けましょう！

In [ ]:
model_config = OmegaConf.create({
    'model': common_config
})

# Then let's attach the sub-module configs
model_config.model.embedding = embedding_config
model_config.model.encoder = encoder_config
model_config.model.decoder = decoder_config

-----この設定を印刷しましょう！

In [ ]:
print(OmegaConf.to_yaml(model_config))

-----待って、OmegaConf はなぜまだ設定ファイルの変数展開値を自動的に埋め込んでくれないの？
これは、OmegaConfが変数補間に対して遅延評価アプローチを採用しているためです。まず必要なフィールドの仮値を設定します（`???`でマークされたフィールド）。その後、事前に解決を強制するには、以下のスニペットを使用できます -

In [ ]:
import copy

In [ ]:
temp_config = copy.deepcopy(model_config)
temp_config.model.vocab_size = 10
temp_config.model.block_size = 4
temp_config.model.n_layer = 1
temp_config.model.n_embd = 32
temp_config.model.n_head = 4

temp_config = OmegaConf.create(OmegaConf.to_container(temp_config, resolve=True))
print(OmegaConf.to_yaml(temp_config))

-----これで設定が完了したので、NeMoモデルのオブジェクトを作成してみましょう！

In [ ]:
# Let's work on a copy of the model config and update it before we send it into the Model.
cfg = copy.deepcopy(model_config)

In [ ]:
# Let's set the values of the config (for some plausible small model)
cfg.model.vocab_size = 100
cfg.model.block_size = 128
cfg.model.n_layer = 1
cfg.model.n_embd = 32
cfg.model.n_head = 4

In [ ]:
print(OmegaConf.to_yaml(cfg))

In [ ]:
# Try to create a model with this config [ERROR CELL]
m = AbstractNeMoGPT(cfg.model)

-----
この NeMo Model に `Abstract` タグを追加した理由は、このモデルをインスタンス化しようとすると、特定のメソッドを実装する必要があることを示すエラーが発生するためです。
1) `setup_training_data` & `setup_validation_data` - すべての NeMo モデルは、トレーニングデータローダーと検証データローダーという 2 つのデータローダーを実装する必要があります。オプションとして、`setup_test_data` メソッドも実装することで、モデル単体での評価をサポートすることもできます。
この制約を設ける理由は、NeMo Modelsが統一された整合性のあるオブジェクトとして設計されているためです。このオブジェクトには、対応するモデルの基盤となるニューラルネットワークの詳細や、モデルの訓練・検証・オプションとしてのテストに使用するデータローダーに関する情報が含まれます。
この処理において、モデルが作成/デシリアライズされた後、モデルをゼロから訓練する、あるいはユーザーが提供した任意のデータセットに対してモデルの微調整/評価を行うには、ユーザー提供のデータセットがこのモデルで使用するデータセット/データローダーがサポートする形式である必要があります！
2) `list_available_models` - これはクラウドからユーザーに事前学習済みNeMoモデルのリストを提供するユーティリティメソッドです。
通常、NeMoモデルはtarファイルに簡単にパッケージ化できます（この種のtarファイルを私たちは「.nemoファイル」と呼んでいます）。これらのtarファイルにはモデルの設定ファイルと、Modelの事前学習済みチェックポイント重みが含まれており、クラウドサービスから簡単にダウンロードできます。
このノートブックでは、この手法を実装しません。
--------最後に、上記のNeMoモデルの具体的な実装を作成してみましょう！

In [ ]:
from nemo.core.classes.common import PretrainedModelInfo

In [ ]:
class BasicNeMoGPT(AbstractNeMoGPT):

  @classmethod
  def list_available_models(cls) -> PretrainedModelInfo:
    return None

  def setup_training_data(self, train_data_config: OmegaConf):
    self._train_dl = None

  def setup_validation_data(self, val_data_config: OmegaConf):
    self._validation_dl = None

  def setup_test_data(self, test_data_config: OmegaConf):
    self._test_dl = None

------では、`BasicNeMoGPT`モデルのオブジェクトを作成してみましょう

In [ ]:
m = BasicNeMoGPT(cfg.model)

## 訓練・検証・テストのステップ設定
上記の `BasicNeMoGPT` モデルは、基本的な PyTorch Lightning モジュールであり、以下の追加機能を備えています：
1) ニューラルタイプチェックのサポート - モデル定義および内部モジュールで定義されたニューラルタイプチェックをサポートします。
2) モデルの保存と復元（単純なケース）を tar ファイルに保存する。
ただし、現在の実装ではこのモデルはPyTorch Lightningの`Trainer`クラスをサポートしていません。このため、このモデルは手動で呼び出すことは可能ですが、PyTorch Lightningフレームワークを使用して簡単に訓練したり評価したりすることはできません。
------
それではこの機能のサポートを追加していきましょう -

In [ ]:
class BasicNeMoGPTWithSteps(BasicNeMoGPT):

    def step_(self, split, batch, batch_idx=None):
        idx, targets = batch
        logits = self(idx=idx)
        loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))
        key = 'loss' if split == 'train' else f"{split}_loss"
        self.log(key, loss)
        return {key: loss}

    def training_step(self, *args, **kwargs):
        return self.step_('train', *args, **kwargs)

    def validation_step(self, *args, **kwargs):
        return self.step_('val', *args, **kwargs)

    def test_step(self, *args, **kwargs):
        return self.step_('test', *args, **kwargs)

    # This is useful for multiple validation data loader setup
    def multi_validation_epoch_end(self, outputs, dataloader_idx: int = 0):
        val_loss_mean = torch.stack([x['val_loss'] for x in outputs]).mean()
        return {'val_loss': val_loss_mean}

    # This is useful for multiple test data loader setup
    def multi_test_epoch_end(self, outputs, dataloader_idx: int = 0):
        test_loss_mean = torch.stack([x['test_loss'] for x in outputs]).mean()
        return {'test_loss': test_loss_mean}

In [ ]:
m = BasicNeMoGPTWithSteps(cfg=cfg.model)

### マルチバリデーションとマルチテストデータローダーの設定
NeMo Primerで説明されているように、NeMoには検証およびテスト段階用の複数のデータローダーが組み込まれています。このようなサポートを追加する方法の一例として、`multi_validation_epoch_end`と`multi_test_epoch_end`のオーバーライドを実装しています。
複数の分散GPUから得られた結果を統合し、エポック終了時に適切に結果を集約することも実務上必須です。NeMoは、たとえ単一デバイスのみで作業する場合でも、結果の正しい統合を厳格に強制します！このケースに対応するため、モデル設計には将来を見据えた設計が組み込まれています！
したがって、NeMoは上記の2つの汎用メソッドを提供し、集約処理をサポートすると同時に複数のデータセットを同時にサポートします！
**注意：既存の`on_validation_epoch_end`と`on_test_epoch_end`実装の先頭に`multi_`を追加するだけで、マルチデータセットとマルチGPUサポートを有効にすることができます！**
------**注意: マルチデータセット機能を無効化するには、`multi_validation_epoch_end` と `multi_test_epoch_end` の代わりに `on_validation_epoch_end` と `on_test_epoch_end` をオーバーライドしてください！**

## オプティマイザ/スケジューラの設定
私たちはMinGPTモデルと同等の機能をほぼ達成しています！ただし、重要な構成要素であるオプティマイザーが欠けています。
すべての NeMo Model には、デフォルトで `setup_optimization()` が実装されています。この関数は、提供されたモデル設定を解析して `optim` と `sched` のサブ設定を取得し、自動的に最適化アルゴリズムとスケジューラを設定します。
もしGPTの訓練がAdam最適化アルゴリズムをすべてのパラメータに適用し、コサイン減衰スケジュールで重み減衰を行うという単純な設定だけで実現可能だったなら、それだけで訓練設定は完了していたでしょう。
-------
ただし、GPTは単なる単純なモデルではなく、より具体的には、重み行列には重み減衰を適用する必要がありますが、バイアス、埋め込み行列、またはLayerNorm層には適用する必要はありません。
このような特殊なケースに対するNemoのサポート機能は不要となり、代わりにPyTorch Lightningの`configure_optimizers`メソッドを使用して同じ処理を実行できます。
-------
注意：NeMoモデルの場合、`configure_optimizers`は`setup_optimization()`への単純な呼び出しとして実装されており、その後生成されたオプティマイザとスケジューラを返すようになっています。したがって、`configure_optimizer`メソッドをオーバーライドし、オプティマイザの作成を手動で管理することが可能です！
NeMoの目的は、一般的なケースに対して実用的なデフォルト値を提供することであり、追加の柔軟性が必要になった場合には、PyTorch LightningまたはPyTorchのnn.Module自体に自動的に切り替わる仕組みになっています！

In [ ]:
class BasicNeMoGPTWithOptim(BasicNeMoGPTWithSteps):

     def configure_optimizers(self):
        """
        This long function is unfortunately doing something very simple and is being very defensive:
        We are separating out all parameters of the model into two buckets: those that will experience
        weight decay for regularization and those that won't (biases, and layernorm/embedding weights).
        We are then returning the PyTorch optimizer object.
        """

        # separate out all parameters to those that will and won't experience weight decay
        decay = set()
        no_decay = set()
        whitelist_weight_modules = (torch.nn.Linear, )
        blacklist_weight_modules = (torch.nn.LayerNorm, torch.nn.Embedding)
        for mn, m in self.named_modules():
            for pn, p in m.named_parameters():
                fpn = '%s.%s' % (mn, pn) if mn else pn # full param name

                if pn.endswith('bias'):
                    # all biases will not be decayed
                    no_decay.add(fpn)
                elif pn.endswith('weight') and isinstance(m, whitelist_weight_modules):
                    # weights of whitelist modules will be weight decayed
                    decay.add(fpn)
                elif pn.endswith('weight') and isinstance(m, blacklist_weight_modules):
                    # weights of blacklist modules will NOT be weight decayed
                    no_decay.add(fpn)

        # special case the position embedding parameter in the root GPT module as not decayed
        no_decay.add('embedding.pos_emb')

        # validate that we considered every parameter
        param_dict = {pn: p for pn, p in self.named_parameters()}
        inter_params = decay & no_decay
        union_params = decay | no_decay
        assert len(inter_params) == 0, "parameters %s made it into both decay/no_decay sets!" % (str(inter_params), )
        assert len(param_dict.keys() - union_params) == 0, "parameters %s were not separated into either decay/no_decay set!" \
                                                    % (str(param_dict.keys() - union_params), )

        # create the pytorch optimizer object
        optim_groups = [
            {"params": [param_dict[pn] for pn in sorted(list(decay))], "weight_decay": self.cfg.optim.weight_decay},
            {"params": [param_dict[pn] for pn in sorted(list(no_decay))], "weight_decay": 0.0},
        ]
        optimizer = torch.optim.AdamW(optim_groups, lr=self.cfg.optim.lr, betas=self.cfg.optim.betas)
        return optimizer


In [ ]:
m = BasicNeMoGPTWithOptim(cfg=cfg.model)

-----それでは、最適化アルゴリズムの設定を行いましょう！

In [ ]:
OmegaConf.set_struct(cfg.model, False)

optim_config = OmegaConf.create({
    'lr': 3e-4,
    'weight_decay': 0.1,
    'betas': [0.9, 0.95]
})

cfg.model.optim = optim_config

OmegaConf.set_struct(cfg.model, True)

## データセット/データローダーの設定
その結果、MinGPTの実装をほぼ完全に再現することができました。
注意：NeMoモデルには、少なくとも訓練ステップと検証ステップのために、データセットとデータローダーを読み込むためのすべてのロジックを含める必要があります。
これまでは回避するために空の実装を一時的に提供してきましたが、今こそそれを実装しましょう！
-------
**データセットに関する注意**: 以下では、オリジナルの[char-rnnリポジトリ](https://github.com/karpathy/char-rnn)で公開されている非常に小規模なデータセット`tiny_shakespeare`を使用した例を示します。実際には任意のテキストコーパスを使用できます。minGPTで推奨されているデータセットは、http://mattmahoney.net/dc/textdata.htmlで入手可能です。

### データセットの作成
NeMoはニューラル型チェックをサポートしており、データセットにも適用可能です！ほとんどの場合、インポート方法が若干変更されるだけで、`collate_fn`の処理方法に1つの違いがあるだけです。
minGPTからデータセット情報を貼り付けるだけで、変更が必要なのはたった2箇所だけです！
-----この例では、HuggingFaceの`nlp`が提供するデータセットの上に薄いサブクラスを記述していきます！

In [ ]:
from nemo.core import Dataset
from torch.utils import data
from torch.utils.data.dataloader import DataLoader

In [ ]:
class TinyShakespeareDataset(Dataset):

  def __init__(self, data_path, block_size, crop=None, override_vocab=None):

      # load the data and crop it appropriately
      with open(data_path, 'r') as f:
          if crop is None:
              data = f.read()
          else:
              f.seek(crop[0])
              data = f.read(crop[1])

      # build a vocabulary from data or inherit it
      vocab = sorted(list(set(data))) if override_vocab is None else override_vocab

      # Add UNK
      special_tokens = ['<PAD>', '<UNK>']  # We use just <UNK> and <PAD> in the call, but can add others.
      if not override_vocab:
        vocab = [*special_tokens, *vocab]  # Update train vocab with special tokens

      data_size, vocab_size = len(data), len(vocab)
      print('data of crop %s has %d characters, vocab of size %d.' % (str(crop), data_size, vocab_size))
      print('Num samples in dataset : %d' % (data_size // block_size))

      self.stoi = { ch:i for i,ch in enumerate(vocab) }
      self.itos = { i:ch for i,ch in enumerate(vocab) }
      self.block_size = block_size
      self.vocab_size = vocab_size
      self.data = data
      self.vocab = vocab
      self.special_tokens = special_tokens

  def __len__(self):
      return len(self.data) // self.block_size

  def __getitem__(self, idx):
      # attempt to fetch a chunk of (block_size + 1) items, but (block_size) will work too
      chunk = self.data[idx*self.block_size : min(len(self.data), (idx+1)*self.block_size + 1)]
      # map the string into a sequence of integers
      ixes = [self.stoi[s] if s in self.stoi else self.stoi['<UNK>'] for s in chunk ]
      # if stars align (last idx and len(self.data) % self.block_size == 0), pad with <PAD>
      if len(ixes) < self.block_size + 1:
          assert len(ixes) == self.block_size # i believe this is the only way this could happen, make sure
          ixes.append(self.stoi['<PAD>'])
      dix = torch.tensor(ixes, dtype=torch.long)
      return dix[:-1], dix[1:]

  @property
  def output_types(self):
    return {
        'input': NeuralType(('B', 'T'), Index()),
        'target': NeuralType(('B', 'T'), LabelsType())
    }

------ここまでは何も変更する必要がありませんでした。では、型チェックはどのように行われるのでしょうか？
NeMoは`collate`関数の実装自体の中で型チェックを行います！この場合、Dataset内で`collate_fn`をオーバーライドする必要はありませんが、もしオーバーライドする必要がある場合は、**NeMoでは代わりにプライベートメソッド`_collate_fn`をオーバーライドする必要があります**。
その後、データローダーを若干の修正で使用することができます！
**また、Dataset用の`input_types`を実装する必要はありません。これらはモデルへの入力を生成する役割を担っているためです！**

-----以下のコードベース[char-rnn](https://github.com/karpathy/char-rnn)からTiny Shakespeareデータセットを準備しましょう。

In [ ]:
import os

In [ ]:
if not os.path.exists('tiny-shakespeare.txt'):
  !wget https://raw.githubusercontent.com/jcjohnson/torch-rnn/master/data/tiny-shakespeare.txt

In [ ]:
!head -n 5 tiny-shakespeare.txt

In [ ]:
train_dataset = TinyShakespeareDataset('tiny-shakespeare.txt', cfg.model.block_size, crop=(0,         int(1e6)))
val_dataset   = TinyShakespeareDataset('tiny-shakespeare.txt', cfg.model.block_size, crop=(int(1e6), int(50e3)), override_vocab=train_dataset.vocab)
test_dataset  = TinyShakespeareDataset('tiny-shakespeare.txt', cfg.model.block_size, crop=(int(1.05e6), int(100e3)), override_vocab=train_dataset.vocab)

### モデルにおけるデータセット/データローダーサポートの設定
これでデータローダーが正常に動作することが確認できました。次はこれをモデル自体の一部として統合しましょう！
これを行うには、NeMo Modelの3つの特殊属性を使用します：`self._train_dl`、`self._validation_dl`、および`self._test_dl`です。DataLoaderを構築したら、これら3つの変数にDataLoaderを割り当ててください。
マルチデータローダーのサポートも同様です！NeMoが自動的に複数のデータローダーの管理を処理してくれます！

In [ ]:
class NeMoGPT(BasicNeMoGPTWithOptim):

  def _setup_data_loader(self, cfg):
    if self.vocab is None:
      override_vocab = None
    else:
      override_vocab = self.vocab

    dataset = TinyShakespeareDataset(
        data_path=cfg.data_path,
        block_size=cfg.block_size,
        crop=tuple(cfg.crop) if 'crop' in cfg else None,
        override_vocab=override_vocab
    )

    if self.vocab is None:
      self.vocab = dataset.vocab

    return DataLoader(
        dataset=dataset,
        batch_size=cfg.batch_size,
        shuffle=cfg.shuffle,
        collate_fn=dataset.collate_fn,  # <-- this is necessary for type checking
        pin_memory=cfg.pin_memory if 'pin_memory' in cfg else False,
        num_workers=cfg.num_workers if 'num_workers' in cfg else 0
    )

  def setup_training_data(self, train_data_config: OmegaConf):
    self.vocab = None
    self._train_dl = self._setup_data_loader(train_data_config)

  def setup_validation_data(self, val_data_config: OmegaConf):
    self._validation_dl = self._setup_data_loader(val_data_config)

  def setup_test_data(self, test_data_config: OmegaConf):
    self._test_dl = self._setup_data_loader(test_data_config)


### データセット/データローダー設定の作成
このモデルを設定する最後の手順は、モデル設定ファイル内に `train_ds`、`validation_ds`、`test_ds` の各設定を追加することです！

In [ ]:
OmegaConf.set_struct(cfg.model, False)

# Set the data path and update vocabular size
cfg.model.data_path = 'tiny-shakespeare.txt'
cfg.model.vocab_size = train_dataset.vocab_size

OmegaConf.set_struct(cfg.model, True)

In [ ]:
train_ds = OmegaConf.create({
    'data_path': '${model.data_path}',
    'block_size': '${model.block_size}',
    'crop': [0, int(1e6)],
    'batch_size': 64,
    'shuffle': True,
})

validation_ds = OmegaConf.create({
    'data_path': '${model.data_path}',
    'block_size': '${model.block_size}',
    'crop': [int(1e6), int(50e3)],
    'batch_size': 4,
    'shuffle': False,
})

test_ds = OmegaConf.create({
    'data_path': '${model.data_path}',
    'block_size': '${model.block_size}',
    'crop': [int(1.05e6), int(100e3)],
    'batch_size': 4,
    'shuffle': False,
})

In [ ]:
# Attach to the model config
OmegaConf.set_struct(cfg.model, False)

cfg.model.train_ds = train_ds
cfg.model.validation_ds = validation_ds
cfg.model.test_ds = test_ds

OmegaConf.set_struct(cfg.model, True)

In [ ]:
# Let's see the config now !
print(OmegaConf.to_yaml(cfg))

In [ ]:
# Let's try creating a model now !
model = NeMoGPT(cfg=cfg.model)

-----すべてのデータローダーが正常に動作しています！やったー！

# モデルの評価 - エンドツーエンドで実施！
データローダーの設定が完了したら、残るはモデルの訓練とテストのみです！このモデルに必要なコンポーネントの大半は既に揃っています - 訓練用、検証用、テスト用データローダー、最適化アルゴリズム、そして訓練・検証・テストの各ステップを実行する型チェック済みの順方向ステップです！
ただし、GPTモデルをゼロから訓練することがこの入門の目的ではないため、代わりにランダムな初期重みを用いてモデルを数ステップ分だけテストすることで、簡易的な動作チェックを行いましょう。
上記により、以下が保証されます：
1) データローダーは意図した通りに動作している
2) 型チェックシステムにより、ニューラルモジュールが順方向処理を正しく実行していることが保証されます。
3) 損失計算が行われ、モデルのエンドツーエンド実行が可能となり、最終的にPyTorch Lightningをサポートします。

In [ ]:
if torch.cuda.is_available():
  accelerator = 'gpu'
else:
  accelerator = 'cpu'

trainer = ptl.Trainer(devices=1, accelerator=accelerator, limit_test_batches=1.0)

In [ ]:
trainer.test(model)

# モデルの保存と復元
NeMoは内部的にモデル構成情報、モデルチェックポイント、およびパラメータを管理しています。
NeMoが上記の一般的なガイドラインに従っている限り、`save_to`と`restore_from`メソッドを使用してモデルの保存と復元が可能です！

In [ ]:
model.save_to('gpt_model.nemo')

In [ ]:
!ls -d -- *.nemo

In [ ]:
temp_model = NeMoGPT.restore_from('gpt_model.nemo')

In [ ]:
# [ERROR CELL]
temp_model.setup_test_data(temp_model.cfg.test_ds)

-----
うーん、今回はそう簡単ではなかったようですね。非自明なモデルには非自明な問題がつきもんですよ！
注意：NeMoGPTモデルでは、`setup_train_data`ステップ内でself.vocabを設定します。ただし、この設定は訓練データセットによって生成された語彙に依存しています... そして、これはモデル復元時には**復元されません**（`setup_train_data`を明示的に呼び出さない限り！）。
外部データファイルを作成することで保存/復元機能をサポートし、この機能はNeMoでもサポートされています！NeMoの`register_artifact` APIを使用して、外部ファイルを.nemoチェックポイントに添付できるようにします。

In [ ]:
class NeMoGPTv2(NeMoGPT):

  def setup_training_data(self, train_data_config: OmegaConf):
    self.vocab = None
    self._train_dl = self._setup_data_loader(train_data_config)

    # Save the vocab into a text file for now
    with open('vocab.txt', 'w') as f:
      for token in self.vocab:
        f.write(f"{token}<SEP>")

    # This is going to register the file into .nemo!
    # When you later use .save_to(), it will copy this file into the tar file.
    self.register_artifact('vocab_file', 'vocab.txt')

  def setup_validation_data(self, val_data_config: OmegaConf):
    # This is going to try to find the same file, and if it fails,
    # it will use the copy in .nemo
    vocab_file = self.register_artifact('vocab_file', 'vocab.txt')

    with open(vocab_file, 'r') as f:
      vocab = []
      vocab = f.read().split('<SEP>')[:-1]  # the -1 here is for the dangling <SEP> token in the file
      self.vocab = vocab

    self._validation_dl = self._setup_data_loader(val_data_config)

  def setup_test_data(self, test_data_config: OmegaConf):
    # This is going to try to find the same file, and if it fails,
    # it will use the copy in .nemo
    vocab_file = self.register_artifact('vocab_file', 'vocab.txt')

    with open(vocab_file, 'r') as f:
      vocab = []
      vocab = f.read().split('<SEP>')[:-1]  # the -1 here is for the dangling <SEP> token in the file
      self.vocab = vocab

    self._test_dl = self._setup_data_loader(test_data_config)


In [ ]:
# Let's try creating a model now !
model = NeMoGPTv2(cfg=cfg.model)

In [ ]:
# Now let's try to save and restore !
model.save_to('gpt_model.nemo')

In [ ]:
temp_model = NeMoGPTv2.restore_from('gpt_model.nemo')

In [ ]:
temp_model.setup_multiple_test_data(temp_model.cfg.test_ds)

In [ ]:
if torch.cuda.is_available():
  accelerator = 'gpu'
else:
  accelerator = 'cpu'

trainer = ptl.Trainer(devices=1, accelerator=accelerator, limit_test_batches =1.0)

In [ ]:
trainer.test(model)

------これで完了です！これで、外部語彙ファイルがある場合でも、モデルのシリアライズとデシリアライズが問題なく行えるようになりました！